# Juliaで固有値問題を精度保証付き数値計算したい


## 固有値問題とは

固有値問題は与えられた行列の固有値および固有ベクトルを求めること。つまり、
$
\newcommand{\C}{\mathbb{C}}
\newcommand{\IC}{\mathbb{IC}}
\newcommand{\bX}{\bar{X}}
$

$$
Ax=\lambda x,\quad A\in \C^{n\times n}
$$

を満たす $\lambda\in \C$, $x\in \C^{n\times n}$ ($x\neq 0$) を求める問題を指す。

Juliaで固有値と固有ベクトルを求める際には、`LinearAlgebra`パッケージに含まれている関数`eigen`・`eigvals`・`eigvecs`で求めることができる。

|関数| 役割 |
|:-----------|:----------|
| eigen()    | 固有値・固有ベクトル両方  |
| eigvals()    | 固有値  |
| eigvecs()  | 固有ベクトル |

多分、Lapackの何かを呼び足してるはず。

In [1]:
using LinearAlgebra
val, vec  = eigen([1 2; 2 1])

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
2-element Array{Float64,1}:
 -1.0
  3.0
vectors:
2×2 Array{Float64,2}:
 -0.707107  0.707107
  0.707107  0.707107

In [2]:
val#固有値

2-element Array{Float64,1}:
 -1.0
  3.0

In [3]:
vec#固有ベクトル

2×2 Array{Float64,2}:
 -0.707107  0.707107
  0.707107  0.707107

In [4]:
eigvals([1 2; 2 1])#固有値

2-element Array{Float64,1}:
 -1.0
  3.0

In [5]:
eigvecs([1 2; 2 1])#固有ベクトル

2×2 Array{Float64,2}:
 -0.707107  0.707107
  0.707107  0.707107

## 密行列の固有値問題に対する精度保証

密行列とは、行列内のほとんどの要素が $0$ でない行列のことを指す。ここでは、行列$A\in\C^{n\times n}$ $(i=1,2,\cdots,n)$ の全ての固有値に対する精度保証アルゴリズムを考える。 

### 実装したいアルゴリズム
(1)
ある行列 $A$ の全ての近似固有ベクトル $\bX\in\C^{n\times n}$ $(i=1,2,\cdots,n)$ を求める。

(2)
$A\cdot\bX$ の包含 $C\in\IC^{n\times n}$ を求める。

(3)
区間連立1次方程式 $\bX G=C$ の解集合の包含 $G\supset \bX^{-1}\cdot C$ ($G\in\IC^{n\times n}$) を求める。

(4)
$G\in\IC^{n\times n}$ に対してゲルシュゴリンの定理を適用することを考える。

#### (1) 行列 $A$ の全ての近似固有ベクトル $\bX\in\C^{n\times n}$ $(i=1,2,\cdots,n)$ を求める。

行列 $A\in\C^{n\times n}$ $(i=1,2,\cdots,n)$ が対角化可能であるとするとき、$\lambda_i$ に対する固有ベクトル$x^{(i)}\in\C^{n}$ $(i=1,2,\cdots,n)$ を並べた行列

$$
X :=[x^{(1)},x^{(2)},\cdots,x^{(n)}]\in\C^{n\times n}
$$

によって、$A$ は

$$
X^{-1}AX = D = \rm{diag}(\lambda_1,\lambda_2,\cdots,\lambda_n)
$$

と対角化される。

In [6]:
using LinearAlgebra

n =100;
A = randn(n,n);

λ, X = eigen(A);

In [7]:
λ #固有値

100-element Array{Complex{Float64},1}:
 -9.354993607755423 - 3.10521804510489im
 -9.354993607755423 + 3.10521804510489im
 -8.829914109169227 + 0.0im
 -8.596852246470185 - 1.420381695818213im
 -8.596852246470185 + 1.420381695818213im
 -7.736385006247817 - 4.386708899636766im
 -7.736385006247817 + 4.386708899636766im
 -7.288022105164716 - 3.0070994784994345im
 -7.288022105164716 + 3.0070994784994345im
 -6.957330762988619 - 1.9038291222117576im
 -6.957330762988619 + 1.9038291222117576im
 -6.688403708534512 - 4.062016699665395im
 -6.688403708534512 + 4.062016699665395im
                    ⋮
  6.424818025552053 - 0.7526839287969654im
  6.424818025552053 + 0.7526839287969654im
  6.684710107899111 - 2.2067326219374404im
  6.684710107899111 + 2.2067326219374404im
   7.06321456373899 - 5.976599344387202im
   7.06321456373899 + 5.976599344387202im
    8.0111505899481 + 0.0im
  8.642183684411435 - 2.8700248930404797im
  8.642183684411435 + 2.8700248930404797im
  9.587731644661712 - 2.76728105018

In [8]:
X #固有ベクトル

100×100 Array{Complex{Float64},2}:
  -0.0324332-0.0676809im    -0.0324332+0.0676809im   …    -0.123008+0.0im
  -0.0256079+0.073751im     -0.0256079-0.073751im          0.020085+0.0im
   0.0166231+0.160937im      0.0166231-0.160937im        -0.0877203+0.0im
   -0.221685-0.0im           -0.221685+0.0im             -0.0825351+0.0im
  0.00644233+0.0717538im    0.00644233-0.0717538im         0.300167+0.0im
  -0.0549602-0.006093im     -0.0549602+0.006093im    …     0.119811+0.0im
   -0.137734+0.0202734im     -0.137734-0.0202734im        0.0253535+0.0im
    0.160319-0.0596742im      0.160319+0.0596742im         0.064853+0.0im
   0.0486162-0.040065im      0.0486162+0.040065im         -0.116457+0.0im
 -0.00727075+0.00947815im  -0.00727075-0.00947815im      -0.0671906+0.0im
   -0.126487-0.10555im       -0.126487+0.10555im     …   -0.0373159+0.0im
  -0.0181367-0.00657198im   -0.0181367+0.00657198im       -0.126041+0.0im
  0.00204521+0.0281492im    0.00204521-0.0281492im        -0.085803+0.0im
   

しかし、実際のところ上記の数値計算で得られる固有ベクトル $X$ は近似の値 $\bX\in \C^{n\times n}$ (以下、近似固有ベクトルという) である。しかし、近似固有ベクトルを並べた行列を使って、対角化したものについては、対角優位性が期待できる。この行列を $G$ とすると、

$$
G := \bX^{-1}A\bX
$$

となり、行列 $G$ の対角成分は行列 $A$ の固有値の近似となる。

また、対角優位性とは、ある行列$A=(a_{ij})$について、

$$
\left|a_{i i}\right|>\sum_{{j} \colon j \neq i}\left|a_{i j}\right| \quad(i=1,2, \ldots, n)
$$

であること。つまり、行列のどの対角成分の絶対値も、それと同じ列にある非対角成分の和よりも大きい時。

In [9]:
G = inv(X)*A*X; #G
norm(diag(G) - λ,Inf) # Gの対角成分は固有値λにほぼ一致する

5.311895280018592e-14

#### (2) $A\cdot\bX$ の包含 $C\in\IC^{n\times n}$ を求める。

``方法1``各演算を区間演算に変更した方式(参考：[interval_dot-mul.ipynb](https://www.risk.tsukuba.ac.jp/~takitoshi/tutorial/interval_dot-mul.html "タイトル"))

In [10]:
using IntervalArithmetic, BenchmarkTools

iA = map(Interval, A);
iX = map(Interval, X);

@btime C1 = $iA * $iX;

  82.239 ms (8 allocations: 312.91 KiB)


In [11]:
@show max(maximum(radius.(real(C1[:]))),maximum(radius.(imag(C1[:]))))

LoadError: UndefVarError: C1 not defined

``方法2``BLASを使う。int_num_linalg.jlの呼び出し。この方が早い。

In [12]:
using BenchmarkTools
include("IntervalLinearAlgebra.jl"); # int_mulを使用する
# @time C1 = mm_comp_interval_alt(iA,iX);
@time C = int_mul(A,X); # A, X: complex matrices (not interval)
@time C = int_mul(A,X); # A, X: complex matrices (not interval)
@time C = int_mul(A,X); # A, X: complex matrices (not interval)


  1.273539 seconds (4.65 M allocations: 222.398 MiB, 3.12% gc time)
  0.157668 seconds (80.04 k allocations: 4.366 MiB)
  0.164723 seconds (80.04 k allocations: 4.366 MiB)


In [13]:
max(maximum(radius.(real(C[:]))),maximum(radius.(imag(C[:]))))

9.059419880941277e-14

In [14]:
real(C)

100×100 Array{Interval{Float64},2}:
  [0.093248, 0.0932481]       …  [-1.26511, -1.2651]
  [0.468574, 0.468575]            [0.206569, 0.20657]
  [0.344236, 0.344237]           [-0.902183, -0.902182]
  [2.07386, 2.07387]             [-0.848854, -0.848853]
  [0.162543, 0.162544]            [3.08715, 3.08716]
  [0.495231, 0.495232]        …   [1.23223, 1.23224]
  [1.35145, 1.35146]              [0.260754, 0.260755]
 [-1.68509, -1.68508]             [0.666997, 0.666998]
 [-0.579215, -0.579214]          [-1.19774, -1.19773]
  [0.0974495, 0.0974496]         [-0.69104, -0.691039]
  [0.855526, 0.855527]        …  [-0.383786, -0.383785]
  [0.149261, 0.149262]           [-1.2963, -1.29629]
  [0.0682766, 0.0682767]         [-0.882464, -0.882463]
   ⋮                          ⋱  
  [1.03076, 1.03077]              [1.06538, 1.06539]
 [-0.452326, -0.452325]          [-0.98697, -0.986969]
 [-0.221006, -0.221005]       …   [0.456314, 0.456315]
 [-1.89666, -1.89665]            [-1.30269, -1.30268]
 [-1

In [15]:
imag(C)

100×100 Array{Interval{Float64},2}:
  [0.733866, 0.733867]     …  [-2.22508e-308, 2.22508e-308]
 [-0.610422, -0.610421]       [-2.22508e-308, 2.22508e-308]
 [-1.55719, -1.55718]         [-2.22508e-308, 2.22508e-308]
  [0.68838, 0.688381]         [-2.22508e-308, 2.22508e-308]
 [-0.691261, -0.69126]        [-2.22508e-308, 2.22508e-308]
  [0.227663, 0.227664]     …  [-2.22508e-308, 2.22508e-308]
  [0.238036, 0.238037]        [-2.22508e-308, 2.22508e-308]
  [0.060425, 0.0604251]       [-2.22508e-308, 2.22508e-308]
  [0.223843, 0.223844]        [-2.22508e-308, 2.22508e-308]
 [-0.0660908, -0.0660907]     [-2.22508e-308, 2.22508e-308]
  [1.38019, 1.3802]        …  [-2.22508e-308, 2.22508e-308]
  [0.117799, 0.1178]          [-2.22508e-308, 2.22508e-308]
 [-0.269687, -0.269686]       [-2.22508e-308, 2.22508e-308]
   ⋮                       ⋱  
 [-0.3235, -0.323499]         [-2.22508e-308, 2.22508e-308]
 [-0.0929236, -0.0929235]     [-2.22508e-308, 2.22508e-308]
  [0.82915, 0.829151]      …  [-2

#### (3 )区間連立1次方程式 $\hat XG=C$ の解集合の包含 $G\supset\hat X^{-1}\cdot C$ を求める。

In [16]:
function verifylss_ib(A,ib) # verify the solution element-wisely
    b = mid.(real(ib)) + mid.(imag(ib))*im;
    x̄ = A\b;
    n = length(x̄);
    R = inv(A);
    #########
    #C_mid, C_rad = mm_ufp(R,A);
    G = Matrix{Float64}(I, n, n) - int_mul(R,A);
    α = opnorm(G,Inf)# Interval arithmetic
    #########
    if α < 1
        x̄ = map(Interval,x̄);
        r = A*x̄ - ib; # Interval arithmetic
        Rr = R*r;
        err = abs.(Rr) + supremum(norm(Rr,Inf))/(1-α)*(abs.(G)*ones(n)); # Interval arithmetic
    else
        println("Oh my way, verification is failed...")
        err = nan;
    end
    return x = (real(x̄) .± supremum.(err)) + im*(imag(x̄) .± supremum.(err))
end

verifylss_ib (generic function with 1 method)

In [17]:
G = similar(C);
for i = 1:n
    G[:,i] = verifylss_ib(X,C[:,i]);
end
real(G)

100×100 Array{Interval{Float64},2}:
 [-9.355, -9.35499]           …  [-8.01304e-13, 7.8056e-13]
 [-1.51816e-12, 1.50535e-12]     [-8.00194e-13, 7.78468e-13]
 [-1.14852e-12, 1.14896e-12]     [-8.39477e-13, 8.21206e-13]
 [-2.17584e-12, 2.17487e-12]     [-1.12967e-12, 1.13731e-12]
 [-2.17559e-12, 2.17473e-12]     [-1.13155e-12, 1.14369e-12]
 [-2.32439e-12, 2.32365e-12]  …  [-1.17874e-12, 1.19363e-12]
 [-2.32585e-12, 2.32249e-12]     [-1.18564e-12, 1.19771e-12]
 [-2.72926e-12, 2.73121e-12]     [-1.40096e-12, 1.42835e-12]
 [-2.73712e-12, 2.72649e-12]     [-1.40478e-12, 1.42781e-12]
 [-2.13365e-12, 2.12841e-12]     [-1.10685e-12, 1.09101e-12]
 [-2.13276e-12, 2.12938e-12]  …  [-1.10352e-12, 1.09056e-12]
 [-4.26999e-12, 4.27706e-12]     [-2.21585e-12, 2.15889e-12]
 [-4.26203e-12, 4.28501e-12]     [-2.20227e-12, 2.19006e-12]
   ⋮                          ⋱  
 [-5.52117e-12, 5.51834e-12]     [-3.08927e-12, 3.08584e-12]
 [-5.50826e-12, 5.52845e-12]     [-3.08194e-12, 3.09883e-12]
 [-3.45936e-12, 

In [18]:
[real(diag(G)), λ]

2-element Array{Array{T,1} where T,1}:
 Interval{Float64}[[-9.355, -9.35499], [-9.355, -9.35499], [-8.82992, -8.82991], [-8.59686, -8.59685], [-8.59686, -8.59685], [-7.73639, -7.73638], [-7.73639, -7.73638], [-7.28803, -7.28802], [-7.28803, -7.28802], [-6.95734, -6.95733]  …  [6.68471, 6.68472], [6.68471, 6.68472], [7.06321, 7.06322], [7.06321, 7.06322], [8.01115, 8.01116], [8.64218, 8.64219], [8.64218, 8.64219], [9.58773, 9.58774], [9.58773, 9.58774], [10.2847, 10.2848]]
 Complex{Float64}[-9.354993607755423 - 3.10521804510489im, -9.354993607755423 + 3.10521804510489im, -8.829914109169227 + 0.0im, -8.596852246470185 - 1.420381695818213im, -8.596852246470185 + 1.420381695818213im, -7.736385006247817 - 4.386708899636766im, -7.736385006247817 + 4.386708899636766im, -7.288022105164716 - 3.0070994784994345im, -7.288022105164716 + 3.0070994784994345im, -6.957330762988619 - 1.9038291222117576im  …  6.684710107899111 - 2.2067326219374404im, 6.684710107899111 + 2.2067326219374404im, 7.063214563

#### 4) $G\in\IC^{n\times n}$ に対してゲルシュゴリンの定理を適用することを考える。


## ゲルシュゴリンの包含定理
行列 $A=(A_{ij})\in \C^{n\times n}$ について、$A$ の全ての固有値 $\lambda_i$ は、$\bigcup_{1\le i \le n}  U_i$ の内部に存在する。

$$
A\subseteq\bigcup_{1\le i \le n}  U_i 
$$

ただし、

$$
U_i = \left\{z\in\mathbb{C}:|z-a_{ii}|\le \sum_{j\neq i}|a_{ij}|\right\}
$$

である。円盤領域 $U_i$ をゲルシュゴリン円板といい、$A$ が強い優対角性を持つとき、$A$ の対角成分が $A$ の固有値の良い近似となる。今回の場合、$G=(g_{ij})_{1\le i,j\le n}\in\IC^{n\times n}$ の対角成分 $(g_{ii})_{1\le i\le n}$ を $(c_{i})_{1\le i\le n}\in \IC^{n}$ とし、各成分の半径を $\mathrm{rad}(c_i)$ とすると、

$$
\begin{array}{ll}
\mbox{ゲルシュゴリン円の中心:}&\mathrm{mid}(c_i)\\
\mbox{ゲルシュゴリン円の半径:}&r_i=\sum_{j\neq i}\rm{mag}(g_{ij})+\rm{rad}(c_{i})\\
\end{array}
$$

となる。

In [19]:
import IntervalArithmetic: mag, radius, mid
function mag(v::Complex{Interval{T}}) where T# mag function for complex interval vectors
    abs_v = abs(v);
    return max(abs_v.lo,abs_v.hi)
end

function radius(v::Complex{Interval{T}}) where T# mag function for complex interval vectors
    return sqrt(interval(radius(real(v)))^2 + interval(radius(imag(v)))^2)
end

function mid(v::Complex{Interval{T}}) where T# mag function for complex interval vectors
    return mid(real(v)) + mid(imag(v))*im
end

mid (generic function with 7 methods)

In [20]:
r = zeros(n);
c = diag(G);
# G_org = G;
# for i ∈ 1:n
#     G[i,i] = 0;
# end
for i = 1:n
    for j = 1:n
        if i != j
            r[i] += mag(G[i,j]);
        end
    end
    r[i] += mag(radius.(c[i]));
end
center = mid.(c);

In [21]:
iλ = (real(center) .± r) + im*(imag(center) .± r);
[real(iλ) imag(iλ)]

100×2 Array{Interval{Float64},2}:
 [-9.355, -9.35499]    [-3.10522, -3.10521]
 [-9.355, -9.35499]     [3.10521, 3.10522]
 [-8.82992, -8.82991]  [-1.5204e-10, 1.5204e-10]
 [-8.59686, -8.59685]  [-1.42039, -1.42038]
 [-8.59686, -8.59685]   [1.42038, 1.42039]
 [-7.73639, -7.73638]  [-4.38671, -4.3867]
 [-7.73639, -7.73638]   [4.3867, 4.38671]
 [-7.28803, -7.28802]  [-3.0071, -3.00709]
 [-7.28803, -7.28802]   [3.00709, 3.0071]
 [-6.95734, -6.95733]  [-1.90383, -1.90382]
 [-6.95734, -6.95733]   [1.90382, 1.90383]
 [-6.68841, -6.6884]   [-4.06202, -4.06201]
 [-6.68841, -6.6884]    [4.06201, 4.06202]
   ⋮                   
  [6.42481, 6.42482]   [-0.752684, -0.752683]
  [6.42481, 6.42482]    [0.752683, 0.752684]
  [6.68471, 6.68472]   [-2.20674, -2.20673]
  [6.68471, 6.68472]    [2.20673, 2.20674]
  [7.06321, 7.06322]   [-5.9766, -5.97659]
  [7.06321, 7.06322]    [5.97659, 5.9766]
  [8.01115, 8.01116]   [-3.50918e-10, 3.50909e-10]
  [8.64218, 8.64219]   [-2.87003, -2.87002]
  [8.64218, 8.642

In [22]:
sum(λ .∈ iλ)

100

In [23]:
center

100-element Array{Complex{Float64},1}:
 -9.354993607755416 - 3.1052180451048947im
 -9.354993607755416 + 3.1052180451048947im
 -8.829914109169218 + 9.247764451284438e-18im
 -8.596852246470188 - 1.4203816958181976im
 -8.596852246470188 + 1.4203816958182043im
 -7.736385006247786 - 4.386708899636768im
 -7.736385006247785 + 4.386708899636766im
 -7.288022105164693 - 3.0070994784994287im
 -7.288022105164693 + 3.0070994784994283im
 -6.957330762988644 - 1.903829122211765im
 -6.957330762988642 + 1.9038291222117634im
 -6.688403708534512 - 4.062016699665401im
 -6.688403708534514 + 4.062016699665399im
                    ⋮
  6.424818025552055 - 0.7526839287969693im
  6.424818025552048 + 0.7526839287969844im
  6.684710107899086 - 2.206732621937441im
  6.684710107899084 + 2.20673262193744im
  7.063214563739001 - 5.976599344387201im
  7.063214563738992 + 5.976599344387202im
   8.01115058994812 - 4.55701832400787e-15im
  8.642183684411458 - 2.8700248930405im
   8.64218368441146 + 2.870024893040492im
  

In [24]:
r

100-element Array{Float64,1}:
 1.953799849767863e-10
 1.9536578211897714e-10
 1.5203988546298948e-10
 2.8000427662466046e-10
 2.799556503764111e-10
 2.952270832930534e-10
 2.9510944431098485e-10
 3.500235520876405e-10
 3.4992355200693e-10
 2.711870472964528e-10
 2.711713790523422e-10
 5.416159327673304e-10
 5.412996700347579e-10
 ⋮
 7.1685165985529e-10
 7.167875476509642e-10
 4.483507430050791e-10
 4.483386415574522e-10
 2.912886353227739e-10
 2.91270955720974e-10
 3.509132225021435e-10
 2.747080236095069e-10
 2.7465486045619395e-10
 2.2749299994649791e-10
 2.2746118225035116e-10
 2.1260035206528936e-10

※11/25↓
https://hsugawa8651.github.io/memojuliav064/ch13.html#%E3%82%AC%E3%82%A6%E3%82%B9%E5%B9%B3%E9%9D%A2-1
を参考に図示しようとしたが、Plotsが上手く作動せずにエラー。

In [40]:
using PyPlot
using Plots

WARNING: using Plots.center in module Main conflicts with an existing identifier.

In [47]:
using PyPlot
zs = [ exp(im*t) for t in 0:pi/18:2pi ]

plot( real.(zs), imag.(zs), ".")
xlabel("real part")
ylabel("imag part")
axhline(0, lw=0.5, color="k")
axvline(0, lw=0.5, color="k")
plt[:axes]()[:set_aspect]("equal")

LoadError: Cannot convert String to series data for plotting

Cannot convert String to series data for plotting

Stacktrace:
 [1] error(::String) at ./error.jl:33
 [2] _prepare_series_data(::String) at /Users/K/.julia/packages/RecipesPipeline/uPBKQ/src/series.jl:8
 [3] _series_data_vector(::String, ::Dict{Symbol,Any}) at /Users/K/.julia/packages/RecipesPipeline/uPBKQ/src/series.jl:27
 [4] macro expansion at /Users/K/.julia/packages/RecipesPipeline/uPBKQ/src/series.jl:146 [inlined]
 [5] apply_recipe(::AbstractDict{Symbol,Any}, ::Type{RecipesPipeline.SliceIt}, ::Any, ::Any, ::Any) at /Users/K/.julia/packages/RecipesBase/AN696/src/RecipesBase.jl:282
 [6] _process_userrecipes!(::Any, ::Any, ::Any) at /Users/K/.julia/packages/RecipesPipeline/uPBKQ/src/user_recipe.jl:36
 [7] recipe_pipeline!(::Any, ::Any, ::Any) at /Users/K/.julia/packages/RecipesPipeline/uPBKQ/src/RecipesPipeline.jl:70
 [8] _plot!(::Plots.Plot, ::Any, ::Any) at /Users/K/.julia/packages/Plots/vsE7b/src/plot.jl:172
 [9] plot(::Any, ::Vararg{Any,N} where N; kw::Any) at /Users/K/.julia/packages/Plots/vsE7b/src/plot.jl:58
 [10] plot(::Any, ::Any, ::Vararg{Any,N} where N) at /Users/K/.julia/packages/Plots/vsE7b/src/plot.jl:52
 [11] top-level scope at In[26]:3
 [12] include_string(::Function, ::Module, ::String, ::String) at ./loading.jl:1091

In [42]:
# 開始値:ステップ:終了値
x = -5.0:0.1:5.0
# 関数
f(x) = 1 ./ (1 .+ exp.(-x))

y = f(x)

using Plots
plot(x, y)
# 画像保存
savefig("Sigmoid.png")

In [43]:

plot!(index, max_rad, yscale=:log10,
    xlabel       ="Real",                #X軸のラベル
    ylabel       ="Imaginary",   #Y軸のラベル
    xlims        =(0,41),             #X軸の範囲
    title="gershgorin",       #タイトル
    linewidth    =2,                  #線幅
    seriestype = :scatter,            #点プロットに　
    size         =(400,300),          #プロットのサイズ
)

LoadError: UndefVarError: index not defined

In [44]:
zs = [ complex(0,0), complex(1,0), complex(1,1), complex(0,1), complex(0,0)]

plot( real.(zs), imag.(zs), ".-")
xlabel("real part")
ylabel("imag part")
xlim(-2,2)
ylim(-2,2)
axhline(0, lw=0.5, color="k")
axvline(0, lw=0.5, color="k")
plt[:axes]()[:set_aspect]("equal")

LoadError: Cannot convert String to series data for plotting

In [45]:
using PyPlot
zs = [ complex(0,0), complex(1,0), complex(1,1), complex(0,1), complex(0,0)]

plot( real.(zs), imag.(zs), ".-")
xlabel("real part")
ylabel("imag part")
xlim(-2,2)
ylim(-2,2)
axhline(0, lw=0.5, color="k")
axvline(0, lw=0.5, color="k")
plt[:axes]()[:set_aspect]("equal")

LoadError: Cannot convert String to series data for plotting

In [50]:
using PyPlot
b=1
cs=linspace(-2,2)
ds=complex(b.*b-4cs)
x1s=(b+sqrt.(ds))/2;
x2v=cs./x1s;
plot(cs, real.(x1s), "b-", label="x1, real part")
plot(cs, imag.(x1s), "b:", label="x1, imag part")
plot(cs, real.(x2v), "g-", label="x2, real part")
plot(cs, imag.(x2v), "g:", label="x2, imag part")
legend()
xlabel("c")
xlim(-2.5,2.5)
ylim(-2.5,2.5)
axhline(0, lw=0.5, color="k")
axvline(0, lw=0.5, color="k")
plt[:axes]()[:set_aspect]("equal")

LoadError: UndefVarError: linspace not defined

対角優位
http://www.math.ritsumei.ac.jp/~yasutomi/jugyo/Numerical_Analysis/note5.pdf

計算途中がでる
http://www.cas.cmc.osaka-u.ac.jp/~paoon/misc/julia/post/cg/


http://infsup.jp/saito/materials/na19exer.pdf